In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import pandas as pd
from scipy.stats import gamma
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [14]:
from dfsmc.projection import projection
from dfsmc.contest import Contest
from dfsmc.lineup import Lineup
from dfsdata import interface as db


contest = Contest.Contest(contest_id=166872333, db_interface=db.DFSDBInterface())
players = contest.get_draftables()
players = players.rename(columns={'name': 'name_display', 'week': 'week_num'})

projector = projection.TrivialProjector(2024, 2)
upcoming_games = projector.populate_upcoming_matchups(contest.draft_group_id)
# print(upcoming_games.head(10))
print(projector.player_game_data.sort_values(by='draftkings_points', ascending=False).head(10))
projections = projector.get_projections(players)
print(projections)
cov = projector.read_covariance()
print(cov)

        name_display  week_num team_name_abbr pos_game  rush_att  rush_yds_per_att  rush_td  two_pt_md  pass_cmp  pass_yds_per_cmp  pass_td  pass_int  fumbles  fumbles_rec_td  all_td  pass_adj_net_yds_per_att  pass_att  pass_sacked  pass_air_yds_per_att  pass_blitzed  pass_hurried  pass_on_target_pct  pass_play_action  pass_poor_throw_pct  pass_pressured_pct  pass_rpo  pass_tgt_yds_per_att  pocket_time  rush_scrambles_yds_per_att   rec  rec_yds_per_rec  rec_td  touches  catch_pct  targets  rec_yds_per_tgt  rec_adot  rec_air_yds_per_rec  rec_drop_pct  rec_yac_per_rec  rush_yds_bc_per_rush  draftkings_points        date game_location opp_name_abbr          cover duration game_day_of_week  game_num   game_result ou_result  over_under  plays_defense  plays_offense  points  points_combined  points_diff  points_opp time_of_poss  tot_yds  vegas_line  yds_per_play_defense  yds_per_play_offense
1228  Jauan Jennings         3            SFO       WR       NaN               NaN      NaN        0.

In [15]:
loc_map = {'@': 'away', '': 'home'}
projections['loc'] = projections['game_location'].map(loc_map)
projections['pos_loc'] = [f'{pos}_{loc}' for pos, loc in zip(projections['pos_game'].values, projections['loc'].values)]
print(projections.head(10))

rows = []
for row in cov.index.values:
    print(row)
    ones = projections['pos_loc'] == row
    ones = np.array(ones.values, dtype=int, ndmin=2)
    rows.append(ones)
con_matrix = np.vstack(rows)
print(con_matrix)

      name_display  draftkings_points team_name_abbr pos_game game_location opp_name_abbr  week_num   loc  pos_loc
0  Braxton Berrios                0.0            MIA       WR                         BUF         2  home  WR_home
1    Curtis Samuel                3.5            BUF       WR             @           MIA         2  away  WR_away
2   Dalton Kincaid                2.1            BUF       TE             @           MIA         2  away  TE_away
3      Dawson Knox                3.3            BUF       TE             @           MIA         2  away  TE_away
4    De'Von Achane               23.0            MIA       RB                         BUF         2  home  RB_home
5    Durham Smythe                0.0            MIA       TE                         BUF         2  home  TE_home
6       James Cook               13.3            BUF       RB             @           MIA         2  away  RB_away
7    Jason Sanders               10.0            MIA        K                   